# Relevance channel estimation

In [26]:
# 1 импорт базовых библиотек и конфига
import pathlib

import pandas as pd
import yaml
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.safe_load(f)

In [27]:
# 2 установка переменных

DATA_PATH = ROOT / config['api']['DATA_PATH']

#query = "корейская косметика"
#type_query = "video"

# Для поиска видео с учетом даты:
publishedAfter = '2020-01-01T00:00:00Z'
publishedBefore = '2022-01-01T00:00:00Z'

In [28]:
# 3  загрузка датафреймов

df_old = pd.read_excel(DATA_PATH / 'seminar5' / 'Каналы_вероятностный_отбор.xlsx')

df = pd.read_excel(DATA_PATH / 'seminar5' / 'Channels_for_tree.xlsx')
df = df.drop_duplicates(subset=['id'])
display(df)

,id,snippet.thumbnails.default.url,snippet.title,snippet.description,topicDetails.topicCategories,brandingSettings.channel.keywords,Вероятностный отбор
0,UCU4_NVJnDYE4l9g2sBYin7A,https://yt3.ggpht.com/ytc/AL5GRJXICyEXBVTZ7hP-...,Корейская косметика в Казахстане,"http://eveli.kz\nВы ищите качественную, но нед...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
1,UCGlTQCQauXBoDLMc8tsd78w,https://yt3.ggpht.com/MxltVIlQq7i9emLzY6otvvnE...,Olesia Корейская косметика Atomy,Я - Олеся - ценитель продукции высокого качест...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
2,UCYvPmv0HQi_KEp8yVGnJ1Wg,https://yt3.ggpht.com/ytc/AL5GRJX0NHyYRNC0qTq_...,Корейская косметика LACORE Уракова Мавлюда,Красота и бизнес,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
3,UCvGowJMOfZJ8ozZmOTg5EZQ,https://yt3.ggpht.com/3uaGBOEEMEiKZHwA_akaHRJs...,AVOCADOKOREAN Корейская косметика,Привет! \nРады видеть тебя на нашем канале AVO...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""корейская косметика"" косметика санкт-петербург",0
4,UCd2qVll5J7bOHgj6OJ-6dBQ,https://yt3.ggpht.com/ytc/AL5GRJUVwBK2CRqRfeMQ...,Корейская косметика от А до Я,Хочешь быть в тренде и одним из первых узнават...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
...,...,...,...,...,...,...,...
180,UCWpYE3K5AdGd_dSNu8Qjdfw,https://yt3.ggpht.com/2__LS6CdwAIk0pgSAxUUZ2NN...,POLINA BELONOSOVA,LIFE STYLE BLOG\n\n,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
181,UCaSyRUFXOLYvpnJTD5bOPYg,https://yt3.ggpht.com/1OZ8OeAu0fNGK25VlemO9J3V...,Dariia Nutrition,"Привет, меня зовут Даша и уже более 6 лет я - ...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,нутрициолог здоровье красота iHerb распаковка ...,0
182,UCMPONXyhtLDUZzUg5NbqSWA,https://yt3.ggpht.com/OMZ8ieoOrEswWZ8mJSz1vYHh...,MakKonee,"먹방, 블로그, 한국에서 러시아 여자의 일상생활\nЭто канал о моей ж...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""южная корея"" корейский ""жизнь в корее"" ""перее...",0
183,UC5T_VQrxIsEQ2b8FYtn6bzQ,https://yt3.ggpht.com/5DJKBoE3lxS0rmZfses3aNEh...,Ольга Величко,Приходите на мой инстограм там много интерес...,['https://en.wikipedia.org/wiki/Entertainment'...,NaN,0


In [29]:
# формирование таргета

import numpy as np

df_ids = df_old[df_old['Вероятностный отбор'] == 1]['id'].to_list()


train = df.query('id in @df_ids').copy()
test = df.query('id not in @df_ids').copy()

In [30]:
train

,id,snippet.thumbnails.default.url,snippet.title,snippet.description,topicDetails.topicCategories,brandingSettings.channel.keywords,Вероятностный отбор
4,UCd2qVll5J7bOHgj6OJ-6dBQ,https://yt3.ggpht.com/ytc/AL5GRJUVwBK2CRqRfeMQ...,Корейская косметика от А до Я,Хочешь быть в тренде и одним из первых узнават...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
5,UClfGcpJdnc7ClcovZoalhBw,https://yt3.ggpht.com/ytc/AL5GRJXjRWvsJE8CJBOu...,LuckyCosmetics Корейская косметика,"Всем привет!\nСпасибо, что зашли на канал инте...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""Корейская косметика"" ""корейская косметика отз...",0
7,UC685uZqmc7Bt46cwUZ80WGQ,https://yt3.ggpht.com/qYTqUI7AsVdUuLki_4DZ0BKE...,GOLDYLOCKS - Магазин корейской косметики,МАГАЗИН ЭКО-КОСМЕТИКИ ИЗ КОРЕИ ДЛЯ ВЗРЫВНОГО Р...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""корейская косметика"" ""магазин корейской косме...",0
8,UCSoadOaxY49VhoAvchaw5eg,https://yt3.ggpht.com/ytc/AL5GRJVSRHeb4AIhdqya...,Mark Kaufman,INSTAGRAM @KAUFMANMK,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
9,UCaqikEbNbRJaUv-DKPFPOHQ,https://yt3.ggpht.com/ytc/AL5GRJXT2eSKZkql0lxr...,Корейская косметика,Корейская косметика в Краснодаре.Мы в инстагра...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,1
12,UCYI1vJzDb1rorX8P1_eCT1g,https://yt3.ggpht.com/ytc/AL5GRJV_NNqZTnrof75T...,RAMOSU корейская проф.косметика,"Дорогой подписчик, мы рады приветствовать Вас ...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""корейская косметика"" Ramosu ""косметика Ramosu...",1
24,UCH_fDahVDXSFuRJNCr2hQMg,https://yt3.ggpht.com/ytc/AL5GRJXK-WruYjHOLCKn...,Корейская косметика Интернет-магазин ФУТИ-БЬЮТ...,Корейская косметика от ведущих производителей ...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"футибьюти ""корейская косметика"" ""интернет-мага...",1
28,UCNx9NXtPgcBspEiQLhaYksQ,https://yt3.ggpht.com/ytc/AL5GRJVAcwljyGaHZbH8...,Корейская косметика Chok-Chok,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0
29,UCCQV_DfKzQ1ScZtbWrQFT0g,https://yt3.ggpht.com/W-b6wTUxeAXssYL5Fpnu6We2...,Оксана ЗАБОТА ЗДЕСЬ,Всем привет!❤️\nМеня зовут Оксана. Мне 27.\nЯ ...,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""порядок в доме"" ""хранение на кухне"" уборка ку...",1
32,UCd2bSV8gaxDAsJiVQAnZYUA,https://yt3.ggpht.com/ytc/AL5GRJXz-nvOCTDuaQ9J...,Корейская Косметика,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,"""корейская косметика оптом"" ""корейская космети...",0


In [ ]:
from sklearn.feature_extraction.text import Tfidf